## Importamos los datos

In [1]:
#Load Libraries/Modules
import pandas as pd
import pyodbc as pyodbc
import warnings
warnings.filterwarnings('ignore')

In [2]:
## https://www.datosabiertos.gob.pe/sites/default/files/Listado_Proveedores_12-03.csv
file_name='Listado_Proveedores_12-03.csv'  
print(file_name)
ruta=r'C:/Users/TITAN/Downloads'

file=ruta+'/'+file_name
print(file)

Listado_Proveedores_12-03.csv
C:/Users/TITAN/Downloads/Listado_Proveedores_12-03.csv


In [3]:
df = pd.read_csv(file,error_bad_lines=False,delimiter=';',engine='python',encoding = 'latin-1') 

warnings.filterwarnings('ignore')

In [4]:
'''
import csv
with open(file, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)
'''

"\nimport csv\nwith open(file, 'r') as file:\n    reader = csv.reader(file)\n    for row in reader:\n        print(row)\n"

In [5]:
df.head(5)

,Ubigeo,Fecha-Actualizacion,Departamento,Provincia,Distrito,Ruc,Proveedor,Modalidad,DomicilioLegal,CorreoElectronico,Telefono
0,10201,2021-03-12,AMAZONAS,BAGUA,BAGUA,10167637316,CONSORCIO ALIMENTOS BAGUA,PRODUCTOS,JR. 28 DE JULIO S/N C-02 SECTOR EL TOMAQUE.,consorcioalimentosbagua@gmail.com,NaN
1,10201,2021-03-12,AMAZONAS,BAGUA,BAGUA,10412710156,DELGADO OLANO MARIBEL ROXANA,PRODUCTOS,Jirón La Libertad N° 217,roxana.del@hotmail.com,NaN
2,10201,2021-03-12,AMAZONAS,BAGUA,BAGUA,10427262923,CONSORCIO AMAZONAS,PRODUCTOS,JR. TUMBES N° 288,emontezafarro@gmail.com,NaN
3,10701,2021-03-12,AMAZONAS,UTCUBAMBA,BAGUA GRANDE,20493681274,INDUSTRIAS ALIMENTARIAS SAN JUAN S.A.C.,PRODUCTOS,JR LIMA N°440 MZ E-2 LT3-SEC. LA ESPERANZA,corporativo@indusanperu.com,14609146
4,10702,2021-03-12,AMAZONAS,UTCUBAMBA,CAJARURO,20561337633,JOSFRANC SOCIEDAD ANONIMA CERRADA,PRODUCTOS,JR. COMERCIO 318,JOSFRANCSAC@HOTMAIL.COM,41263102


In [6]:
df.shape

(185, 11)

In [7]:
#detail_column=df.info()
detail_column=df.dtypes
print('-----------------')
type(detail_column)
print(detail_column)

-----------------
Ubigeo                  int64
Fecha-Actualizacion    object
Departamento           object
Provincia              object
Distrito               object
Ruc                     int64
Proveedor              object
Modalidad              object
DomicilioLegal         object
CorreoElectronico      object
Telefono               object
dtype: object


In [8]:
# Convert entire DataFrame to string
df=df.applymap(str)
#print(df.dtypes)

In [9]:
# Limpiamos las columnas
df.columns = df.columns.str.replace('[#,@,&,°]', '')
df.columns = df.columns.str.replace('[(,)]', ' ')
df.columns = df.columns.str.replace('[á]', 'a')
df.columns = df.columns.str.replace('[é]', 'e')
df.columns = df.columns.str.replace('[í]', 'i')
df.columns = df.columns.str.replace('[ó]', 'o')
df.columns = df.columns.str.replace('[ú]', 'u')

df.columns = df.columns.str.replace('[ñ]', 'ni')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
#
df.columns = df.columns.str.lower()
warnings.filterwarnings('ignore')

In [10]:
print(df.dtypes)

ubigeo                 object
fecha_actualizacion    object
departamento           object
provincia              object
distrito               object
ruc                    object
proveedor              object
modalidad              object
domiciliolegal         object
correoelectronico      object
telefono               object
dtype: object


## Conectamos con SQL

In [11]:
#Conexion SQL Server
import pyodbc as pyodbc
sql_conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};\
                            SERVER=localhost;\
                            DATABASE=ACADEMY;\
                            Trusted_Connection=yes') 
cursor = sql_conn.cursor()

In [12]:
#
query_drop_table = """
drop table if exists ACADEMY.dbo.LISTADO_PROVEEDORES
"""
query_drop_table

'\ndrop table if exists ACADEMY.dbo.LISTADO_PROVEEDORES\n'

In [13]:
cursor.execute(query_drop_table)
sql_conn.commit()

In [14]:
#Creamos la tabla
# Si la tabla no existe , se crea :
query_create_table = """
CREATE TABLE dbo.LISTADO_PROVEEDORES (
Ubigeo                varchar(250) NULL,
Fecha_Actualizacion   varchar(250) NULL,
Departamento          varchar(250) NULL,
Provincia             varchar(250) NULL,
Distrito              varchar(250) NULL,
Ruc                   varchar(250) NULL,
Proveedor             varchar(250) NULL,
Modalidad             varchar(250) NULL,
DomicilioLegal        varchar(250) NULL,
CorreoElectronico     varchar(250) NULL,
Telefono              varchar(250) NULL
)
"""

In [15]:
#Creamos la tabla
cursor.execute(query_create_table)
sql_conn.commit()

In [16]:
#Preparamos la tabla para poder cargarla
df_list=df.values.tolist()

In [17]:
#Diseñamos el Query de insert
MY_TABLE = 'LISTADO_PROVEEDORES'
cols = ",".join([str(i) for i in df.columns.tolist()])
#print(cols) // son 49 campos, son 49 ????
insert_to_tmp_tbl_stmt = f"INSERT INTO dbo.{MY_TABLE} VALUES (?,?,?,?,?,?,?,?,?,?,?)"
insert_to_tmp_tbl_stmt

'INSERT INTO dbo.LISTADO_PROVEEDORES VALUES (?,?,?,?,?,?,?,?,?,?,?)'

## Cargamos a SQL

In [18]:
#Cargamos la base de datos a SQL Server
try:
    for record in df_list:
        #print(record)
        cursor.execute(insert_to_tmp_tbl_stmt, record)        
except Exception as e:
    cursor.rollback()
    print(e.value)
    print('transaction rolled back')
else:
    print('df_list inserted successfully')
    cursor.commit()
    #cursor.close()
#finally:
    #if sql_conn.connected == 1:
        #print('connection closed')
        #sql_conn.close()

df_list inserted successfully


In [19]:
#Validamos que la data se encuentre cargada
df_check = pd.read_sql_query('SELECT * FROM LISTADO_PROVEEDORES', sql_conn)

In [20]:
df_check.head(3)

,Ubigeo,Fecha_Actualizacion,Departamento,Provincia,Distrito,Ruc,Proveedor,Modalidad,DomicilioLegal,CorreoElectronico,Telefono
0,10201,2021-03-12,AMAZONAS,BAGUA,BAGUA,10167637316,CONSORCIO ALIMENTOS BAGUA,PRODUCTOS,JR. 28 DE JULIO S/N C-02 SECTOR EL TOMAQUE.,consorcioalimentosbagua@gmail.com,nan
1,10201,2021-03-12,AMAZONAS,BAGUA,BAGUA,10412710156,DELGADO OLANO MARIBEL ROXANA,PRODUCTOS,Jirón La Libertad N° 217,roxana.del@hotmail.com,nan
2,10201,2021-03-12,AMAZONAS,BAGUA,BAGUA,10427262923,CONSORCIO AMAZONAS,PRODUCTOS,JR. TUMBES N° 288,emontezafarro@gmail.com,nan


# Siguientes pasos
1. Descargar los datos de la WEB mediante WebScraping
2. Automatizar el proceso mediando Batch / CMD